In [1]:
%load_ext lab_black

In [2]:
import json
import pandas as pd
import numpy as np
from utils import import_df

In [3]:
def get_days(df):
    # CREAZIONE COLONNA GIORNO E MACCHINA
    df["day"] = 0
    df["machine"] = 0
    counter_measure = 0
    counter_day = 0
    counter_machine = 0
    for i, row in df.iterrows():
        # i 50 giorni sono in un piccolo condominio
        if (counter_measure % 50) == 0:
            counter_measure = 0
            counter_day += 1
        # i 7 giorni sono arbitrari ma penso siano opportuni dato che comunque
        # sono dati appartenenti al medesimo motore in ciclo giornaliero
        if (counter_day % 7) == 0:
            counter_measure = 0
            counter_day = 1
            counter_machine += 1
        df.at[i, "day"] = counter_day
        df.at[i, "machine"] = counter_machine
        counter_measure += 1

    # CREAZIONE COLONNE LABELS
    # per i bearings
    conditions = [
        (df["label"] == "bearings-1") | (df["label"] == "bearings-2"),
        (df["label"] == "working-engine-1") | (df["label"] == "working-engine-2"),
    ]
    values = [1, 0]
    df["bearings"] = np.select(conditions, values)

    # per il resto appena abbiamo i dati bisogna aggungere la riga
    df["rotor"] = 0
    df["electricity"] = 0

    df = df.drop("label", axis=1)

    return df


data_frame = import_df("3-noise-generation-output.json")
data_frame = get_days(data_frame)
data_frame

,s_00,s_01,s_02,s_03,s_04,s_05,s_06,s_07,s_08,s_09,...,vib_z_acc,vib_z_kurt,vib_z_peak,vib_z_vel,current,day,machine,bearings,rotor,electricity
0,-0.265235,-5.048122,-3.631346,-4.439543,-3.357498,-0.204776,-1.318228,-0.339534,-1.400015,-0.246901,...,1.467088,0.164199,0.817187,2.849615,3.974052,1,0,0,0,0
1,1.803906,-7.633756,-5.389268,-1.570934,-2.791938,-1.642577,-1.893539,0.155088,-0.426586,0.362725,...,1.844653,-0.383697,0.384010,0.855245,3.966374,1,0,0,0,0
2,0.034944,-7.383289,-10.814815,-5.313512,-2.078825,-0.999530,-2.935479,-0.510686,-0.604878,-0.327708,...,1.696922,0.437205,0.764138,2.917843,3.971816,1,0,0,0,0
3,-1.236713,-6.816264,-4.414403,-2.289054,-2.182520,-1.306811,-1.128795,0.227098,0.474560,-0.287856,...,1.543970,0.487229,0.668764,2.192881,3.963132,1,0,0,0,0
4,-0.631867,1.805891,-4.057512,-3.777951,-4.524474,-0.794026,-2.293586,0.294561,-0.637668,-0.196524,...,1.837363,0.157348,0.354827,1.877270,3.961272,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31795,1.621144,6.426994,2.831992,0.201274,-0.176498,0.919015,2.139039,0.051557,0.285876,-0.177516,...,0.556325,1.119456,0.995157,0.564188,3.933841,6,105,0,0,0
31796,-0.753628,-2.764779,-0.148560,-2.216969,-0.432313,0.391412,0.198170,-0.161367,-0.208661,0.363186,...,1.191670,1.480539,0.851721,0.213766,3.946284,6,105,0,0,0
31797,-0.224139,-1.600675,-2.785047,0.721778,0.588208,-0.655339,1.173323,-0.078556,-0.200729,-0.444910,...,1.644411,1.433706,1.033311,1.286546,3.932896,6,105,0,0,0
31798,-1.836151,2.152550,-0.564253,0.057216,-0.133246,0.419357,-0.712477,-1.009656,1.032995,-0.294873,...,1.370810,1.055747,0.920297,0.808079,3.916539,6,105,0,0,0


## Extract features

In [4]:
def relevant_df(df, machine_id, label, day):
    data = df[(df.machine == machine_id) & (df.day == day)]
    data = data[label]
    return data


def mean(df, machine_id, data_label, day, **kwargs):
    mean = relevant_df(df, machine_id, data_label, day).mean()
    return mean


def rms(df, machine_id, data_label, day, **kwargs):
    s = relevant_df(df, machine_id, data_label, day) ** 2
    ms = s.mean()
    import math

    rms_ = math.sqrt(ms)
    return rms_


def variance(df, machine_id, data_label, day, **kwargs):
    values = relevant_df(df, machine_id, data_label, day)
    var = values.var()
    return var


def skewness(df, machine_id, data_label, day, **kwargs):
    skew = relevant_df(df, machine_id, data_label, day).skew()
    return skew


def kurtosis(df, machine_id, data_label, day, **kwargs):
    kur = relevant_df(df, machine_id, data_label, day).kurtosis()
    return kur


def max_value(df, machine_id, data_label, day, **kwargs):
    max = relevant_df(df, machine_id, data_label, day).max()
    return max


def quantile(df, machine_id, data_label, day, **kwargs):
    q = kwargs.get("quantile", 0.5)

    df = relevant_df(df, machine_id, data_label, day)
    quantile = df.quantile(q)
    return quantile


def interquantile_range(df, machine_id, data_label, day, **kwargs):
    low = kwargs.get("low", 0.25)
    high = kwargs.get("high", 0.75)

    q_low = quantile(df, machine_id, data_label, day, quantile=low)
    q_high = quantile(df, machine_id, data_label, day, quantile=high)
    iqr = q_high - q_low

    return iqr


def mean_absolute_deviation(df, machine_id, data_label, day, **kwargs):
    mad = relevant_df(df, machine_id, data_label, day).mad()
    return mad


def trimmed_mean(df, machine_id, data_label, day, **kwargs):
    trim = kwargs.get("trim", 0.1)
    df = relevant_df(df, machine_id, data_label, day)
    n = len(df)
    trim_index = int(n * trim)
    trimmed_df = df.iloc[trim_index:-trim_index]
    mean = trimmed_df.mean()
    return mean


def peak_value(df, machine_id, data_label, day, **kwargs):
    abs = relevant_df(df, machine_id, data_label, day).abs()
    peak = abs.max()
    return peak


def crest_factor(df, machine_id, data_label, day, **kwargs):
    peak = peak_value(df, machine_id, data_label, day)
    rms_ = rms(df, machine_id, data_label, day)
    cf = peak / rms_
    return cf

In [6]:
machine_id = 0
extractors = [
    mean,
    rms,
    variance,
    skewness,
    kurtosis,
    max_value,
    peak_value,
    crest_factor,
    quantile,
    interquantile_range,
    mean_absolute_deviation,
    trimmed_mean,
]

df = data_frame

# Example days: [1 2 3 4 5 6]
days = df["day"].unique()
clean_df = df.drop(["day", "machine", "bearings", "rotor", "electricity"], axis=1)
data_labels = list(clean_df.columns)

features = dict()
for extractor in extractors:
    machine_features = pd.DataFrame(index=days, columns=data_labels)
    for label in data_labels:
        for day in days:
            machine_features.at[day, label] = extractor(df, machine_id, label, day)
    features[extractor.__name__] = machine_features


for f, df in features.items():
    print(f)
    print(df)
    print("")

mean
       s_00      s_01      s_02      s_03      s_04      s_05      s_06  \
1 -0.075797 -6.304629 -6.125342 -3.763346 -2.376768 -1.033901 -1.020849   
2  0.302567 -4.739052 -6.794774 -3.577214 -2.027509 -1.031448 -1.145908   
3   0.69255 -5.007701  -6.12639 -4.072779 -2.112284 -1.088299 -0.994412   
4 -0.043008 -5.404912  -6.73458 -3.853967 -2.094819 -1.111142 -1.158376   
5  0.293717 -5.113159 -6.506987 -4.104445 -2.094984 -0.763824 -0.894777   
6  0.286755 -4.760526  -6.50076 -3.557327 -1.964461 -1.015573 -0.960962   

       s_07      s_08      s_09  ...  vib_y_f2  vib_y_f3 vib_y_kurt  \
1 -0.166536 -0.076349 -0.043285  ...  1.105943  0.987576   0.572517   
2  0.008592 -0.302386 -0.080778  ...   1.05997  0.934554   0.552739   
3 -0.032937 -0.130659  0.256489  ...  1.101394  0.918694   0.549691   
4 -0.061505  0.157691  0.012474  ...   1.20994  1.300532   0.527097   
5 -0.079515  0.112587 -0.162707  ...   1.22297  1.256333   0.526531   
6 -0.218471  0.127639 -0.038597  ...  1.048